# 2.0

## How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? 

You merge sorted lists/arrays using merge sorts which is used between the mapper output and the when they go into the reducer in the shuffle stage.

## What is  a combiner function in the context of Hadoop?

A combiner function is a semi-reducer in that it is an optional class that can process the output of the map tasks before it's submitted to the reducer tasks.  This is essentially summarizing the map output before it is transfered to the reducer and is done to limit the data transfered across the network and save on expense since the output from the map task is usually quite large.

## Give an example where it can be used and justify why it should be used in the context of this problem.

As stated in previous answer, this can be used to summarize the mapper results before they are sent over the network to the reducer.  This should be done when the results from the map tasks are large to save expense of all the data transfering accross the network.

## What is the Hadoop shuffle?

The shuffle phase partitions data among the various reducers.  Data from all of the mappers are grouped together by the key and then split among reducers (each reducer get's the values of the same key) and sorted by the key.  Sorting and partitioning can also be done using custom functions for different scenarios.

# 2.1: Counters as a debugging aid 

In [26]:
!curl -L https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0 -o Consumer_Complaints.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 48.5M  100 48.5M    0     0  2290k      0  0:00:21  0:00:21 --:--:-- 2807k


In [27]:
%%writefile mr_cc_counter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        #self.increment_counter('group', 'Num_mapper_calls', 1)
        word = line.split(',')[1]
        #self.increment_counter(word, 'Num_mapper_calls', 1)
        if word != "Product":
            if word == "Debt collection" or word == "Mortgage":
                self.increment_counter(word.lower(), 'Num_mapper_calls', 1)
                yield word.lower(), 1
            else:
                self.increment_counter('other', 'Num_mapper_calls', 1)
                yield "other", 1

    def reducer(self, word, counts):
        self.increment_counter(word, 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_cc_counter.py


In [28]:
%run mr_cc_counter Consumer_Complaints.csv

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.65943

"debt collection"	44372
"mortgage"	125752
"other"	142788


Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_cc_counter.z0018t0.20160710.152048.659438...
Removing temp directory /var/fol

Counters:
debt collection - 44372
mortgage - 125752
other - 142788

# 2.2: Analyze the performance of your Mappers, Combiners and Reducers using Counters

In [29]:
!echo foo foo quux labs foo bar quux > WordCount.txt

In [30]:
%%writefile mr_wc_counter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_wc_counter.py


In [31]:
%run mr_wc_counter WordCount.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f

"bar"	1
"foo"	3
"labs"	1
"quux"	2


Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_wc_counter.z0018t0.20160710.152055.514303...
Removing temp directory /var/fol

The mapper is only called once since there is just 1 line of data, however, the reducer is called 4 times since there are 4 unique words in the line.

In [32]:
%%writefile mr_22_counter.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import csv
 
WORD_RE = re.compile(r"[\w']+")
 
class MRWordFreqCount(MRJob):
    def mapper(self, _, line):
        self.increment_counter('group', 'Num_mapper_calls', 1)
        for line in csv.reader([line]):
            #self.increment_counter(line[3].lower(), 'Num_mapper_calls', 1)
            if line[3] != "Issue":
                yield line[3].lower(), 1
            #else:
            #    print "Find Issue Kai"
        
    def reducer(self, word, counts):
        self.increment_counter('group', 'Num_reducer_calls', 1)
        #self.increment_counter(word, 'Num_reducer_calls', 1)
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordFreqCount.run()

Overwriting mr_22_counter.py


In [33]:
%run mr_22_counter Consumer_Complaints.csv

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136
Creating t

""	4
"account opening, closing, or management"	16205
"account terms and changes"	350
"advertising and marketing"	1193
"application processing delay"	243
"application, originator, mortgage broker"	8625
"applied for loan/did not receive money"	139
"apr or interest rate"	3431
"arbitration"	168
"balance transfer fee"	95
"balance transfer"	502
"bankruptcy"	222
"billing disputes"	6938
"billing statement"	1220
"can't contact lender"	221
"can't repay my loan"	1647
"can't stop charges to bank account"	131
"cash advance fee"	104
"cash advance"	136
"charged bank acct wrong day or amt"	71
"charged fees or interest i didn't expect"	807
"closing/cancelling account"	2795
"collection debt dispute"	904
"collection practices"	1003
"communication tactics"	8671
"cont'd attempts collect debt not owed"	17972
"convenience checks"	75
"credit card protection / debt protection"	1343
"credit decision / underwriting"	2774
"credit determination"	1490
"credit line increase/decrease"	1149
"credit monitoring or ident

Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152055.590136...
Removing temp directory /var/fol

The mapper is called 312913 times since there are that many lines of data, however, the reducer is only called 79 times since there are 79 unique issues.

# 2.2.1

In [34]:
from mr_22_counter import MRWordFreqCount

modelStats={}

mr_job = MRWordFreqCount(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('HW2model_2_2_1.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152103.074677
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152103.074677
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152103.074677
Creating t

 4
account opening, closing, or management 16205
account terms and changes 350
advertising and marketing 1193
application processing delay 243
application, originator, mortgage broker 8625
applied for loan/did not receive money 139
apr or interest rate 3431
arbitration 168
balance transfer fee 95
balance transfer 502
bankruptcy 222
billing disputes 6938
billing statement 1220
can't contact lender 221
can't repay my loan 1647
can't stop charges to bank account 131
cash advance fee 104
cash advance 136
charged bank acct wrong day or amt 71
charged fees or interest i didn't expect 807
closing/cancelling account 2795
collection debt dispute 904
collection practices 1003
communication tactics 8671
cont'd attempts collect debt not owed 17972
convenience checks 75
credit card protection / debt protection 1343
credit decision / underwriting 2774
credit determination 1490
credit line increase/decrease 1149
credit monitoring or identity protection 1453
credit reporting company's investigation 48

In [35]:
%%writefile mr_221_sorter.py

from mrjob.job import MRJob
from functools import cmp_to_key
import locale

def fillinValues(values):
    l = []
    for v in values:
        l.append(v)
    return sorted(l, key=cmp_to_key(locale.strcoll))

class MR50MostFreqWords(MRJob):
        
    def __init__(self, *args, **kwargs):
        super(MR50MostFreqWords, self).__init__(*args, **kwargs)
        self.modelStats = {}
        self.stack1 = []
        self.stack2 = []
        self.stack3 = []
        
    def jobconf(self):
        orig_jobconf = super(MR50MostFreqWords, self).jobconf()        
        custom_jobconf = {'mapred.reduce.tasks': '1'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
        
    def mapper(self, _, line):
        l = line.split('\t')
        yield("%08d"%int(l[1]), l[0])
        
    def reducer(self, key, values):
        self.modelStats[key] = fillinValues(values)
        while len(self.stack1) != 0 and self.stack1[-1] < key:
            self.stack2.append(self.stack1.pop())
        if self.stack2 != [] or len(self.stack1) < 50:
            self.stack1.append(key)
        while len(self.stack1) < 50 and self.stack2 != []:
            self.stack1.append(self.stack2.pop())
        if self.stack2 != []:
            self.stack2.pop()

        while len(self.stack3) != 0 and self.stack3[-1] > key:
            self.stack2.append(self.stack3.pop())
        if self.stack2 != [] or len(self.stack3) < 10:
            self.stack3.append(key)   
        while len(self.stack3) < 10 and self.stack2 != []:
            self.stack3.append(self.stack2.pop())
        if self.stack2 != []:
            self.stack2.pop()           


    def reducer_final(self):
        wordCount1 = 0
        wordCount3 = 0
        print "50 Most Frequent Words:"
        for v in self.stack1:
            for word in self.modelStats[v]:
                wordCount1 += 1
                if wordCount1 <= 50:
                    print "\"" + word + "\" happens " + v + " times ranking " + str(wordCount1) + " most frequent."       
                    
        print "\n\n10 Least Frequent Words:"
        for v in self.stack3:
            for word in self.modelStats[v]:
                wordCount3 += 1
                if wordCount3 <= 10:
                    print "\"" + word + "\" happens " + v + " times ranking " + str(wordCount3) + " least frequent."


if __name__ == '__main__':
    MR50MostFreqWords.run()

Overwriting mr_221_sorter.py


In [36]:
%run mr_221_sorter HW2model_2_2_1.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr

50 Most Frequent Words:
"loan modification,collection,foreclosure" happens 00070487 times ranking 1 most frequent.
"loan servicing, payments, escrow account" happens 00036767 times ranking 2 most frequent.
"incorrect information on credit report" happens 00029069 times ranking 3 most frequent.
"cont'd attempts collect debt not owed" happens 00017972 times ranking 4 most frequent.
"account opening, closing, or management" happens 00016205 times ranking 5 most frequent.
"deposits and withdrawals" happens 00010555 times ranking 6 most frequent.
"communication tactics" happens 00008671 times ranking 7 most frequent.
"application, originator, mortgage broker" happens 00008625 times ranking 8 most frequent.
"disclosure verification of debt" happens 00007655 times ranking 9 most frequent.
"billing disputes" happens 00006938 times ranking 10 most frequent.
"other" happens 00006273 times ranking 11 most frequent.
"problems caused by my funds being low" happens 00005663 times ranking 12 most fre

Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152110.783293...
Removing temp directory /var/fol

## 2.2.2

In [37]:
from mr_22_counter import MRWordFreqCount

modelStats={}

mr_job = MRWordFreqCount(args=['Consumer_Complaints.csv'])
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('HW2model_2_2_3.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152110.856300
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_22_counter.z0018t0.20160710.152110.856300
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr

 4
account opening, closing, or management 16205
account terms and changes 350
advertising and marketing 1193
application processing delay 243
application, originator, mortgage broker 8625
applied for loan/did not receive money 139
apr or interest rate 3431
arbitration 168
balance transfer fee 95
balance transfer 502
bankruptcy 222
billing disputes 6938
billing statement 1220
can't contact lender 221
can't repay my loan 1647
can't stop charges to bank account 131
cash advance fee 104
cash advance 136
charged bank acct wrong day or amt 71
charged fees or interest i didn't expect 807
closing/cancelling account 2795
collection debt dispute 904
collection practices 1003
communication tactics 8671
cont'd attempts collect debt not owed 17972
convenience checks 75
credit card protection / debt protection 1343
credit decision / underwriting 2774
credit determination 1490
credit line increase/decrease 1149
credit monitoring or identity protection 1453
credit reporting company's investigation 48

In [38]:
%%writefile mr_221_sorter.py

from mrjob.job import MRJob
from functools import cmp_to_key
import locale

def fillinValues(values):
    l = []
    for v in values:
        l.append(v)
    return sorted(l, key=cmp_to_key(locale.strcoll))

class MR50MostFreqWords(MRJob):
        
    def __init__(self, *args, **kwargs):
        super(MR50MostFreqWords, self).__init__(*args, **kwargs)
        self.modelStats = {}
        self.stack1 = []
        self.stack2 = []
        self.stack3 = []
        
    def jobconf(self):
        orig_jobconf = super(MR50MostFreqWords, self).jobconf()        
        custom_jobconf = {'mapred.reduce.tasks': '3'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
        
    def mapper(self, _, line):
        l = line.split('\t')
        yield("%08d"%int(l[1]), l[0])
        
    def reducer(self, key, values):
        self.modelStats[key] = fillinValues(values)
        while len(self.stack1) != 0 and self.stack1[-1] < key:
            self.stack2.append(self.stack1.pop())
        if self.stack2 != [] or len(self.stack1) < 50:
            self.stack1.append(key)
        while len(self.stack1) < 50 and self.stack2 != []:
            self.stack1.append(self.stack2.pop())
        if self.stack2 != []:
            self.stack2.pop()

        while len(self.stack3) != 0 and self.stack3[-1] > key:
            self.stack2.append(self.stack3.pop())
        if self.stack2 != [] or len(self.stack3) < 10:
            self.stack3.append(key)   
        while len(self.stack3) < 10 and self.stack2 != []:
            self.stack3.append(self.stack2.pop())
        if self.stack2 != []:
            self.stack2.pop()           


    def reducer_final(self):
        wordCount1 = 0
        wordCount3 = 0
        print "50 Most Frequent Words:"
        for v in self.stack1:
            for word in self.modelStats[v]:
                wordCount1 += 1
                if wordCount1 <= 50:
                    print "\"" + word + "\" happens " + v + " times ranking " + str(wordCount1) + " most frequent"       
                    
        print "\n\n10 Least Frequent Words:"
        for v in self.stack3:
            for word in self.modelStats[v]:
                wordCount3 += 1
                if wordCount3 <= 10:
                    print "\"" + word + "\" happens " + v + " times ranking " + str(wordCount3) + " least frequent"


if __name__ == '__main__':
    MR50MostFreqWords.run()

Overwriting mr_221_sorter.py


In [39]:
%run mr_221_sorter HW2model_2_2_3.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854
Creating temp director

50 Most Frequent Words:
"shopping for a loan or lease" happens 00000535 times ranking 1 most frequent
"balance transfer" happens 00000502 times ranking 2 most frequent
"managing the line of credit" happens 00000446 times ranking 3 most frequent
"other transaction issues" happens 00000387 times ranking 4 most frequent
"account terms and changes" happens 00000350 times ranking 5 most frequent
"forbearance / workout plans" happens 00000350 times ranking 6 most frequent
"getting a loan" happens 00000291 times ranking 7 most frequent
"money was not available when promised" happens 00000274 times ranking 8 most frequent
"application processing delay" happens 00000243 times ranking 9 most frequent
"privacy" happens 00000240 times ranking 10 most frequent
"bankruptcy" happens 00000222 times ranking 11 most frequent
"can't contact lender" happens 00000221 times ranking 12 most frequent
"arbitration" happens 00000168 times ranking 13 most frequent
"other service issues" happens 00000151 times ra

Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_221_sorter.z0018t0.20160710.152118.348854...
Removing temp directory /var/fol

These results are not what we are looking for because each of the reducers are generating the most/least frequent words from their own data.

# 2.3

In [40]:
!curl -L https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0 -o ProductPurchaseData.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3377k  100 3377k    0     0  1252k      0  0:00:02  0:00:02 --:--:-- 1963k


In [41]:
%%writefile mr_23_uniqueItems.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class ProductPurchase(MRJob):
    def __init__(self, *args, **kwargs):
        super(ProductPurchase, self).__init__(*args, **kwargs)
        self.ReducerCalls = 0
        
    def jobconf(self):
        orig_jobconf = super(ProductPurchase, self).jobconf()        
        custom_jobconf = {'mapred.reduce.tasks': '1'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield word, 1

    def reducer(self, word, counts):
        self.ReducerCalls += 1
        yield word, sum(counts)
        
    def reducer_final(self):
        print "There are " + str(self.ReducerCalls) + " unique items."

if __name__ == '__main__':
    ProductPurchase.run()

Overwriting mr_23_uniqueItems.py


In [42]:
%run mr_23_uniqueItems ProductPurchaseData.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniq

There are 12592 unique items.
"DAI11153"	8
"DAI11223"	155
"DAI11238"	3
"DAI11257"	1
"DAI11261"	6
"DAI11273"	1
"DAI11290"	5
"DAI11299"	2
"DAI11375"	1
"DAI11462"	8
"DAI11541"	5
"DAI11552"	8
"DAI11555"	25
"DAI11582"	1
"DAI11613"	2
"DAI11695"	5
"DAI11707"	1
"DAI11778"	117
"DAI11927"	73
"DAI11946"	1
"DAI11995"	1
"DAI12036"	6
"DAI12139"	1
"DAI12152"	1
"DAI12275"	2
"DAI12437"	16
"DAI12448"	1
"DAI12460"	1
"DAI12521"	78
"DAI12535"	1
"DAI12558"	3
"DAI12566"	3
"DAI12680"	65
"DAI12683"	2
"DAI12698"	1
"DAI12720"	49
"DAI12740"	3
"DAI12770"	3
"DAI12821"	2
"DAI12844"	2
"DAI12853"	2
"DAI12862"	21
"DAI12870"	1
"DAI12879"	1
"DAI12900"	56
"DAI12903"	1
"DAI12919"	9
"DAI12961"	7
"DAI12976"	1
"DAI13045"	1
"DAI13051"	2
"DAI13072"	2
"DAI13117"	1
"DAI13129"	6
"DAI13194"	116
"DAI13234"	1
"DAI13266"	181
"DAI13268"	14
"DAI13285"	2
"DAI13306"	1
"DAI13331"	1
"DAI13440"	5
"DAI13501"	1
"DAI13509"	2
"DAI13519"	1
"DAI13585"	4
"DAI13587"	1
"DAI13616"	57
"DAI13619"	5
"DAI13640"	3
"DAI13709"	6
"DAI13730"	1
"DAI13737"	1
"DA

Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152121.265054...


12592 unique items

In [43]:
%%writefile mr_23_largestBasket.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
 
WORD_RE = re.compile(r"[\w']+")
 
class ProductPurchase(MRJob):
    def __init__(self, *args, **kwargs):
        super(ProductPurchase, self).__init__(*args, **kwargs)
        
    def jobconf(self):
        orig_jobconf = super(ProductPurchase, self).jobconf()        
        custom_jobconf = {'mapred.reduce.tasks': '1'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
    
    def mapper(self, _, line):
        ls = len(WORD_RE.findall(line))
        yield "words", ls

    def reducer(self, word, counts):
        largestBasket = 0
        for v in counts:
            largestBasket = max(largestBasket, v)
        yield "largestBasket", largestBasket           
        

if __name__ == '__main__':
    ProductPurchase.run()

Overwriting mr_23_largestBasket.py


In [44]:
%run mr_23_largestBasket ProductPurchaseData.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048
Creating temp directory /var

"largestBasket"	37


Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.152126.404048...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_largestBasket.z0018t0.20160710.1

Largest basket contains 37 items

In [45]:
from mr_23_uniqueItems import ProductPurchase

modelStats={}

mr_job = ProductPurchase(args=['ProductPurchaseData.txt'])
with mr_job.make_runner() as runner: 
    runner.run()
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        print key, value
        modelStats[key] = value            
    with open('HW2model_2_3_1.txt', 'w') as f:
        for k in modelStats.keys():
            f.writelines( k + "\t"+ str(modelStats[k]) +"\n")

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547
Creating temp directory /var/f

There are 12592 unique items.
DAI11153 8
DAI11223 155
DAI11238 3
DAI11257 1
DAI11261 6
DAI11273 1
DAI11290 5
DAI11299 2
DAI11375 1
DAI11462 8
DAI11541 5
DAI11552 8
DAI11555 25
DAI11582 1
DAI11613 2
DAI11695 5
DAI11707 1
DAI11778 117
DAI11927 73
DAI11946 1
DAI11995 1
DAI12036 6
DAI12139 1
DAI12152 1
DAI12275 2
DAI12437 16
DAI12448 1
DAI12460 1
DAI12521 78
DAI12535 1
DAI12558 3
DAI12566 3
DAI12680 65
DAI12683 2
DAI12698 1
DAI12720 49
DAI12740 3
DAI12770 3
DAI12821 2
DAI12844 2
DAI12853 2
DAI12862 21
DAI12870 1
DAI12879 1
DAI12900 56
DAI12903 1
DAI12919 9
DAI12961 7
DAI12976 1
DAI13045 1
DAI13051 2
DAI13072 2
DAI13117 1
DAI13129 6
DAI13194 116
DAI13234 1
DAI13266 181
DAI13268 14
DAI13285 2
DAI13306 1
DAI13331 1
DAI13440 5
DAI13501 1
DAI13509 2
DAI13519 1
DAI13585 4
DAI13587 1
DAI13616 57
DAI13619 5
DAI13640 3
DAI13709 6
DAI13730 1
DAI13737 1
DAI13776 31
DAI13780 1
DAI13788 213
DAI13865 18
DAI13883 29
DAI13902 233
DAI13943 12
DAI13973 2
DAI14019 17
DAI14125 163
DAI14127 1
DAI14128 21
DAI14

Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_23_uniqueItems.z0018t0.20160710.152127.035547...


 1
GRO78277 3
GRO78320 4
GRO78359 2
GRO78407 76
GRO78441 43
GRO78469 17
GRO78501 14
GRO78505 71
GRO78558 16
GRO78671 2
GRO78700 5
GRO78723 3
GRO78740 5
GRO78762 1
GRO78803 9
GRO78820 2
GRO78847 3
GRO78871 1
GRO78960 2
GRO78961 1
GRO78983 2
GRO79029 5
GRO79033 5
GRO79040 1
GRO79120 8
GRO79141 11
GRO79186 1
GRO79195 4
GRO79204 25
GRO79209 10
GRO79228 18
GRO79239 5
GRO79259 7
GRO79260 1
GRO79275 1
GRO79323 8
GRO79335 16
GRO79451 1
GRO79455 7
GRO79586 1
GRO79612 3
GRO79633 1
GRO79645 3
GRO79718 1
GRO79767 16
GRO79772 2
GRO79833 3
GRO79838 1
GRO79858 1
GRO79870 16
GRO79910 35
GRO80001 81
GRO80002 3
GRO80025 1
GRO80047 30
GRO80063 2
GRO80075 3
GRO80094 58
GRO80099 1
GRO80119 1
GRO80138 3
GRO80142 2
GRO80154 1
GRO80159 14
GRO80204 1
GRO80205 1
GRO80211 1
GRO80241 1
GRO80299 1
GRO80355 1
GRO80374 2
GRO80520 2
GRO80621 6
GRO80692 39
GRO80702 30
GRO80726 1
GRO80793 11
GRO80809 1
GRO80832 2
GRO80833 3
GRO80863 2
GRO80897 41
GRO80898 1
GRO80907 1
GRO80926 1
GRO80942 7
GRO80955 25
GRO80995 12
GRO81

In [46]:
%%writefile mr_223_sorter.py

from mrjob.job import MRJob
from functools import cmp_to_key
import locale

def fillinValues(values):
    l = []
    for v in values:
        l.append(v)
    return sorted(l, key=cmp_to_key(locale.strcoll))

class MR50MostFreqWords(MRJob):
        
    def __init__(self, *args, **kwargs):
        super(MR50MostFreqWords, self).__init__(*args, **kwargs)
        self.modelStats = {}
        self.stack1 = []
        self.stack2 = []
        
    def jobconf(self):
        orig_jobconf = super(MR50MostFreqWords, self).jobconf()        
        custom_jobconf = {'mapred.reduce.tasks': '1'}
        combined_jobconf = orig_jobconf
        combined_jobconf.update(custom_jobconf)
        self.jobconf = combined_jobconf
        return combined_jobconf
        
    def mapper(self, _, line):
        l = line.split('\t')
        yield("%04d"%int(l[1]), l[0])
        
    def reducer(self, key, values):
        while len(self.stack1) != 0 and self.stack1[-1] < key:
            self.stack2.append(self.stack1.pop())
        if self.stack2 != [] or len(self.stack1) < 50:
            self.stack1.append(key)
            self.modelStats[key] = fillinValues(values)
        while len(self.stack1) < 50 and self.stack2 != []:
            self.stack1.append(self.stack2.pop())
        if self.stack2 != []:
            del self.modelStats[self.stack2.pop()]


    def reducer_final(self):
        wordCount = 0
        print "50 Most Frequent Items:"
        for v in self.stack1:
            for item in self.modelStats[v]:
                wordCount += 1
                if wordCount <= 50:
                    print "\"" + item + "\" happens " + v + " times ranking " + str(wordCount) + " most frequent"       

if __name__ == '__main__':
    MR50MostFreqWords.run()

Overwriting mr_223_sorter.py


In [47]:
%run mr_223_sorter.py HW2model_2_3_1.txt

No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
No configs found; falling back on auto-configuration
Creating temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.2016

50 Most Frequent Items:
"DAI62779" happens 6667 times ranking 1 most frequent
"FRO40251" happens 3881 times ranking 2 most frequent
"ELE17451" happens 3875 times ranking 3 most frequent
"GRO73461" happens 3602 times ranking 4 most frequent
"SNA80324" happens 3044 times ranking 5 most frequent
"ELE32164" happens 2851 times ranking 6 most frequent
"DAI75645" happens 2736 times ranking 7 most frequent
"SNA45677" happens 2455 times ranking 8 most frequent
"FRO31317" happens 2330 times ranking 9 most frequent
"DAI85309" happens 2293 times ranking 10 most frequent
"ELE26917" happens 2292 times ranking 11 most frequent
"FRO80039" happens 2233 times ranking 12 most frequent
"GRO21487" happens 2115 times ranking 13 most frequent
"SNA99873" happens 2083 times ranking 14 most frequent
"GRO59710" happens 2004 times ranking 15 most frequent
"GRO71621" happens 1920 times ranking 16 most frequent
"FRO85978" happens 1918 times ranking 17 most frequent
"GRO30386" happens 1840 times ranking 18 most freq

Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/folders/8f/z0f9xjsn7m7_zv37bc65t2f91vw1s7/T/mr_223_sorter.z0018t0.20160710.152132.299939...
Removing temp directory /var/fol